In [2]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as smf

from scipy import stats
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)
import patsy
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.svm import SVC
%matplotlib inline


In [19]:
np.random.seed(1)
#Uncomment and set the path to the csv file below
#file_path_jump = r'D:\visual studio code\project\EEN210-main\EEN210-main\repeated_jumping_20240205_155636.csv'
file_path_updown =r'D:\visual studio code\project\EEN210-main\EEN210-main\sitting up and down 2min.csv'
file_path_walk=r'D:\visual studio code\project\EEN210-main\EEN210-main\walk_2min_data_20240212_154122.csv'
file_path_fall=r'D:\visual studio code\project\EEN210-main\EEN210-main\5bagtoss_20240211_203142.csv'


walk_data = pd.read_csv(file_path_walk)
updown_data = pd.read_csv(file_path_updown)
#stand_data=pd.read_csv(file_path_stand)
fall_data=pd.read_csv(file_path_fall)

walk_data["label"]=0
updown_data["label"]=1
fall_data["label"]=2
updown_data.head(100)


,acceleration_x,acceleration_y,acceleration_z,gyroscope_x,gyroscope_y,gyroscope_z,timestamp,label
0,-0.8142,-0.1233,-0.0125,3.5095,-6.8665,-2.0370,2024-02-12 15:49:19.455,1
1,-0.8208,-0.1052,-0.0088,6.0196,-8.7204,-3.0060,2024-02-12 15:49:19.455,1
2,-0.8091,-0.0869,-0.0156,7.8964,-10.4675,-3.7079,2024-02-12 15:49:19.479,1
3,-0.8081,-0.0840,-0.0071,8.1787,-8.5449,-4.3106,2024-02-12 15:49:19.495,1
4,-0.8096,-0.0718,-0.0176,7.3242,-1.0223,-5.5161,2024-02-12 15:49:19.511,1
...,...,...,...,...,...,...,...,...
95,-0.7458,-0.1482,0.0122,-12.3215,38.1546,-28.6942,2024-02-12 15:49:21.167,1
96,-0.9395,-0.1025,0.1331,-4.7455,28.0228,-32.7148,2024-02-12 15:49:21.191,1
97,-0.9331,-0.1370,0.0298,0.7858,8.7280,-20.8969,2024-02-12 15:49:21.207,1
98,-0.7339,-0.1987,-0.0046,-0.9308,4.7531,-9.0637,2024-02-12 15:49:21.231,1


In [21]:
columns = [
        "acceleration_x",
        "acceleration_y",
        "acceleration_z",
        "gyroscope_x",
        "gyroscope_y",
        "gyroscope_z",
    ]

data = pd.concat([walk_data, updown_data, fall_data])

# Preprocess the data
X = data.drop(['timestamp', 'label'], axis=1)  # Features
y = data['label']  # Target labels

# Convert the DataFrame to numpy arrays


In [22]:

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
#print(X_train[0])
normalizer = StandardScaler()

X_train = normalizer.fit_transform(X_train)
X_test = normalizer.transform(X_test)
print(X_train[0])

[-0.25062482  0.09098766  0.58453704 -1.27310754  0.78888969  0.39777123]


In [24]:
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Define the neural network architecture
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  # 3 output classes
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Make predictions
y_pred_probs = model.predict(X_test)
y_pred = tf.argmax(y_pred_probs, axis=1)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Print evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


ModuleNotFoundError: No module named 'tensorflow'

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
import numpy as np

# Define the neural network architecture
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.softmax(out)
        return out

# Function to train the model
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss}")

# Function to evaluate the model
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    print(f"Accuracy on test set: {accuracy}")

# Load your data and preprocess it

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

#Create DataLoader objects
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define hyperparameters
input_size = X_train.shape[1]
hidden_size = 64
num_classes = len(np.unique(y_train))
learning_rate = 0.001
num_epochs = 10

# Create the neural network model
model_NN = NeuralNetwork(input_size, hidden_size, num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_NN.parameters(), lr=learning_rate)

# Train the model
train_model(model_NN, train_loader, criterion, optimizer, num_epochs)

# Evaluate the model
evaluate_model(model_NN, test_loader)


Epoch [1/10], Loss: 168.29856592416763
Epoch [2/10], Loss: 133.96286988258362
Epoch [3/10], Loss: 124.84278094768524
Epoch [4/10], Loss: 121.49272096157074
Epoch [5/10], Loss: 119.7707439661026
Epoch [6/10], Loss: 118.73235285282135
Epoch [7/10], Loss: 117.98577731847763
Epoch [8/10], Loss: 117.47499591112137
Epoch [9/10], Loss: 117.04709595441818
Epoch [10/10], Loss: 116.69656324386597
Accuracy on test set: 0.9244946492271106


In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
import numpy as np

# Define the LSTM model
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Function to train the LSTM model
def train_lstm_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss}")

# Function to evaluate the LSTM model
def evaluate_lstm_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    print(f"Accuracy on test set: {accuracy}")

window_length = 50
num_features = 6  # Number of features in your data

# Define a function to reshape your data
def reshape_data(data, window_length):
    num_samples, num_features = data.shape
    num_windows = num_samples - window_length + 1
    reshaped_data = np.zeros((num_windows, window_length, num_features))
    for i in range(num_windows):
        window = data[i:i+window_length, :]
        reshaped_data[i] = window
    return reshaped_data

# Reshape your data
X_ = reshape_data(X.values, window_length)
print(X_.shape)  # Output should be (num_sequences, time_length, num_features)

y_ = y[window_length-1:] 



X_train_, X_test_, y_train_, y_test_ = train_test_split(X_, y_, test_size=0.3)

# Load your data and preprocess it
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_.values.astype(int), dtype=torch.long)  # Assuming y is a pandas Series
y_test_tensor = torch.tensor(y_test_.values.astype(int), dtype=torch.long)

# Create DataLoader objects
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



# Define hyperparameters
input_size = 6 # Example input size
hidden_size = 64
num_layers = 2
num_classes = 3 # Example number of classes
learning_rate = 0.001
num_epochs = 10

# Create the LSTM model
model_lstm = LSTMClassifier(input_size, hidden_size, num_layers, num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_lstm.parameters(), lr=learning_rate)

# Train the LSTM model
train_lstm_model(model_lstm, train_loader, criterion, optimizer, num_epochs)

# Evaluate the LSTM model
evaluate_lstm_model(model_lstm, test_loader)


(16771, 50, 6)
Epoch [1/10], Loss: 55.75095129292458
Epoch [2/10], Loss: 7.819618345238268
Epoch [3/10], Loss: 7.393310149433091
Epoch [4/10], Loss: 3.996404674369842
Epoch [5/10], Loss: 0.9487196463742293
Epoch [6/10], Loss: 0.6281467087683268
Epoch [7/10], Loss: 0.4861428744334262
Epoch [8/10], Loss: 0.27536201509064995
Epoch [9/10], Loss: 0.17382752822595648
Epoch [10/10], Loss: 0.14423761586658657
Accuracy on test set: 1.0


In [15]:
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_class_weight

def calc_accuracy(pred_y, y):
    """Calculate accuracy."""
    return ((pred_y == y).sum() / len(y)).item()

class LSTMNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, target_size, batch_size=10, dropout=0.5):
        super(LSTMNet, self).__init__()
        self.norm_layer = nn.BatchNorm1d(
            input_dim, affine=True, track_running_stats=False
        )
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, target_size)
        self.dropout = nn.Dropout(p=dropout)
        self.batch_size = batch_size

    def forward(self, input_):
        x = self.norm_layer(input_.permute(0, 2, 1)).permute(0, 2, 1)
        lstm_out, (h, c) = self.lstm(x)
        logits = self.fc(lstm_out[:, -1])
        logits = self.dropout(logits)
        scores = F.softmax(logits, dim=1)
        return scores

    def fit(
        self,
        x_train,
        y_train,
        x_val=None,
        y_val=None,
        epochs=1,
        verbose=False,
        plot=False,
        lr=1e-4,
        weight_decay=5e-5,
    ):
        trainloader = DataLoader(
            TensorDataset(x_train, y_train), batch_size=self.batch_size, shuffle=True
        )
        class_weights = compute_class_weight(
            class_weight="balanced", classes=np.unique(y_train), y=y_train.numpy()
        )
        print(class_weights)
        class_weights = torch.tensor(class_weights, dtype=torch.float)
        criterion = torch.nn.CrossEntropyLoss(weight=class_weights)
        # criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(
            self.parameters(),
            lr=lr,
            weight_decay=weight_decay,
        )

        batch_acc_list = []
        train_acc_list = []
        val_acc_list = []
        batch_loss_list = []
        train_loss_list = []
        val_loss_list = []
        for epoch in range(epochs + 1):
            losses = []
            accuracies = []
            self.train()
            for i, data in enumerate(trainloader, 0):
                inputs, labels = data
                # Training
                optimizer.zero_grad()
                y_pred = self(inputs)
                loss = criterion(y_pred, labels)
                loss.backward()
                optimizer.step()
                losses.append(loss.detach().numpy())
                batch_acc = calc_accuracy(y_pred.argmax(dim=1), labels)
                accuracies.append(batch_acc)
            avg_loss = np.mean(losses)
            avg_accuracy = np.mean(accuracies)

            # Print metrics every 10 epochs
            if verbose and x_val is not None:
                y_pred_train = self(x_train)
                train_loss = criterion(y_pred_train, y_train)
                train_acc = calc_accuracy(y_pred_train.argmax(dim=1), y_train)
                # f1_score = calc_f1_score(y_pred.argmax(dim=1), y_train)

                # Validation
                self.eval()
                y_pred_val = self(x_val)
                val_loss = criterion(y_pred_val, y_val)
                val_acc = calc_accuracy(y_pred_val.argmax(dim=1), y_val)
                # val_f1_score = calc_f1_score(y_pred_val.argmax(dim=1), y_val)

                if epoch % 1 == 0:
                    print(
                        f"Epoch {epoch:>3} | Train Loss: {avg_loss:.3f} | Train Acc:"
                        f" {avg_accuracy*100:>6.2f}% | Val Loss: {val_loss:.2f} | "
                        # f"F1 Score: {f1_score*100:.2f}% | "
                        f"Val Acc: {val_acc*100:.2f}% | ",
                        # f"Val F1 Score: {val_f1_score*100:.2f}%",
                        end="\r",
                    )

                batch_acc_list.append(avg_accuracy)
                train_acc_list.append(train_acc)
                val_acc_list.append(val_acc)
                batch_loss_list.append(avg_loss)
                train_loss_list.append(train_loss.detach().numpy())
                val_loss_list.append(val_loss.detach().numpy())

        if verbose:
            print("\n")

        if plot:
            plt.figure()
            plt.plot(batch_acc_list, label="batch")
            # plt.plot(train_acc_list, label="train")
            plt.plot(val_acc_list, label="val")
            plt.xlabel("epochs")
            plt.title("accuracy")
            plt.legend()

            plt.figure()
            plt.plot(batch_loss_list, label="batch")
            # plt.plot(train_loss_list, label="train")
            plt.plot(val_loss_list, label="val")
            plt.xlabel("epochs")
            plt.title("loss")
            plt.legend()

        if x_val is not None:
            self.eval()
            y_pred_val = self(x_val)
            val_loss = criterion(y_pred_val, y_val)
            val_acc = calc_accuracy(y_pred_val.argmax(dim=1), y_val)

            print(
                classification_report(
                    y_val.numpy(), y_pred_val.detach().argmax(dim=1).numpy()
                )
            )

            return val_acc, val_loss
        
#
window_length = 50
num_features = 6  # Number of features in your data

# Define a function to reshape your data
def reshape_data(data, window_length):
    num_samples, num_features = data.shape
    num_windows = num_samples - window_length + 1
    reshaped_data = np.zeros((num_windows, window_length, num_features))
    for i in range(num_windows):
        window = data[i:i+window_length, :]
        reshaped_data[i] = window
    return reshaped_data

# Reshape your data
X_ = reshape_data(X.values, window_length)
print(X_.shape)  # Output should be (num_sequences, time_length, num_features)

y_ = y[window_length-1:] 



X_train_, X_test_, y_train_, y_test_ = train_test_split(X_, y_, test_size=0.3)

model = LSTMNet(X_.shape[2], hidden_dim=10, target_size=2, batch_size=10, dropout=0.1)
X_train_tensor = torch.tensor(X_train.astype(np.float32))
y_train_tensor = torch.tensor(y_train.values.astype(int))
X_test_tensor = torch.tensor(X_test.astype(np.float32))
y_test_tensor = torch.tensor(y_test.values.astype(int))
model.fit(
    X_train_tensor,
    y_train_tensor,
    X_test_tensor,
    y_test_tensor,
    epochs=1000,
    verbose=True,
    plot=True,
    lr=1e-4,
    weight_decay=1e-5,
)

(16771, 50, 6)
[0.92628432 0.79948394 1.49340436]


RuntimeError: number of dims don't match in permute

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score, train_test_split
import joblib
#import pickle

def save_model(model, filename):
    try:
        joblib.dump(model, filename)
        #pickle.dump(model, filename)
        print("Model saved successfully.")
    except Exception as e:
        print("Error saving model:", e)

def load_model(filename):
    try:
        #model = pickle.load(filename)
        model = joblib.load(filename)
        print("Model loaded successfully.")
        return model
    except Exception as e:
        print("Error loading model:", e)
        return None

def train_model(method, X_train, y_train):
    if method == 'RandomForest':
        model = RandomForestClassifier(n_estimators = 25, class_weight="balanced")
    elif method == 'DecisionTree':
        model = DecisionTreeClassifier()
    elif method == 'SVM':
        model = SVC()
    elif method =='GradientBoost':
        model== GradientBoostingClassifier()
    else:
        raise ValueError("Invalid machine learning method specified.")
    
    try:
        model.fit(X_train, y_train)
        return model
    except Exception as e:
        print("Error training model:", e)
        return None

def predict_label(model, data):
    try:
        label = model.predict(data)
        return label
    except Exception as e:
        print("Error predicting label:", e)
        return None

def predict_label_NN(model, data):
    try:
        # Convert data to tensor
        data_tensor = torch.tensor(data, dtype=torch.float32)
        
        # Make predictions using the model
        with torch.no_grad():
            outputs = model(data_tensor)
            _, predicted = torch.max(outputs, 1)
        
        return predicted.numpy()  # Convert predictions to numpy array
    except Exception as e:
        print("Error predicting label:", e)
        return None

def predict_label_LSTM(model, data):
    try:
        # Convert data to tensor
        data_tensor = torch.tensor(data, dtype=torch.float32)
        
        # Make predictions using the model
        with torch.no_grad():
            outputs = model(data_tensor)
            _, predicted = torch.max(outputs, 1)
        
        return predicted.numpy()  # Convert predictions to numpy array
    except Exception as e:
        print("Error predicting label:", e)
        return None

def calculate_accuracy(true_labels, predicted_labels):
    # Count the occurrences of the target label in the true labels
    correct_target_labels = np.sum(predicted_labels==true_labels)
    print(correct_target_labels)
    
    # Count the occurrences of the target label in the predicted labels
    #correct_predicted_labels = sum((true_labels == target_label) & (predicted_labels == target_label))
    
    # Calculate the accuracy
    if correct_target_labels > 0:
        accuracy = (correct_target_labels / len(predicted_labels)) * 100
    else:
        accuracy = 0
    
    return accuracy



# Example usage:
# Train the model
# model = train_model()
method = 'RandomForest'

trained_model = train_model(method, X_train, y_train)

# Save the trained model
save_model(trained_model, "model.pkl")
save_model(normalizer, "normalizer.pkl")

# Load the trained model
loaded_model = load_model("model.pkl")
loaded_norm = load_model("normalizer.pkl")

# Example prediction
file_path=r'D:\visual studio code\project\EEN210-main\EEN210-main\fall_on_back_data_20240211_195608.csv'

#file_path_jump = r'D:\visual studio code\project\EEN210-main\EEN210-main\repeated_jumping_20240205_155636.csv'
file_path_updown =r'D:\visual studio code\project\EEN210-main\EEN210-main\sitting up and down 2min.csv'
file_path_walk=r'D:\visual studio code\project\EEN210-main\EEN210-main\walk_2min_data_20240212_154122.csv'
#walk_data["label"]=0 updown_data["label"]=1 fall_data["label"]=2

target_label = 2
data = pd.read_csv(file_path)  #choose test file here
X = data.drop(['timestamp', 'label'], axis=1)  # Features
y = data['label']

#normalizer = StandardScaler()

X = loaded_norm.transform(X)

#np.set_printoptions(threshold=np.inf) 
np.set_printoptions(threshold=20) 
# data = ... # Load or generate your data here
label = predict_label(loaded_model, X)
print("Predicted label:", label)
accuracy_label = calculate_accuracy(target_label, label)
print(f"Accuracy for label {target_label}: {accuracy_label:.2f}%")
# Save the trained model
save_model(model_NN, "model_NN.pkl")

# Load the trained model
loaded_model = load_model("model_NN.pkl")
label = predict_label_NN(loaded_model, X)
print("Predicted label:", label)
# Calculate the accuracy for label 2
accuracy_label = calculate_accuracy(target_label, label)
print(f"Accuracy for label {target_label}: {accuracy_label:.2f}%")

window_length = 50
num_features = 6  # Number of features in your data

# Define a function to reshape your data
def reshape_data(data, window_length):
    num_samples, num_features = data.shape
    num_windows = num_samples - window_length + 1
    reshaped_data = np.zeros((num_windows, window_length, num_features))
    for i in range(num_windows):
        window = data[i:i+window_length, :]
        reshaped_data[i] = window
    return reshaped_data

# Reshape your data
X_ = reshape_data(X, window_length)
print(X_.shape)  # Output should be (num_sequences, time_length, num_features)

y_ = y[window_length-1:] 

save_model(model_lstm, "model_lstm.pkl")

# Load the trained model
loaded_model = load_model("model_lstm.pkl")
label = predict_label_LSTM(loaded_model, X_)
print("Predicted label:", label)
# Calculate the accuracy for label 2
accuracy_label = calculate_accuracy(target_label, label)
print(f"Accuracy for label {target_label}: {accuracy_label:.2f}%")

Model saved successfully.
Model saved successfully.
Model loaded successfully.
Model loaded successfully.
Predicted label: [2 2 2 ... 2 2 2]
5224
Accuracy for label 2: 50.62%
Model saved successfully.
Model loaded successfully.
Predicted label: [2 2 2 ... 2 2 2]
5491
Accuracy for label 2: 53.21%
(10271, 50, 6)
Model saved successfully.
Model loaded successfully.
Predicted label: [2 2 2 ... 2 2 2]
6745
Accuracy for label 2: 65.67%
